In [3]:
import numpy as np
import scipy as sp
import suite2p as s2p
import TwoPUtils
import os
from matplotlib import pyplot as plt

%matplotlib inline

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
source_folder  = os.path.join('/media','mplitt','Backup Plus1','2P_Data')
write_folder = os.path.join('/mnt','BigDisk','2P_scratch')


mouse = '4467331.1'

file_list = ({'date': '29_11_2020', 'scene':'YMaze_LNovel','session':1,'scan':3},
             {'date': '30_11_2020', 'scene':'YMaze_LNovel','session':1,'scan':7},
             {'date': '01_12_2020', 'scene':'YMaze_LNovel','session':1,'scan':4},
             {'date': '02_12_2020', 'scene':'YMaze_LNovel','session':1,'scan':3},
             {'date': '03_12_2020', 'scene':'YMaze_LNovel','session':1,'scan':8},
             {'date': '04_12_2020', 'scene':'YMaze_LNovel','session':1,'scan':2},
             {'date': '05_12_2020', 'scene':'YMaze_LNovel','session':1,'scan':2}, 
             {'date':'05_12_2020','scene':'YMaze_RewardReversal', 'session':1,'scan':4},
             {'date': '06_12_2020', 'scene':'YMaze_RewardReversal','session':2,'scan':1},
             {'date': '07_12_2020', 'scene':'YMaze_RewardReversal','session':1,'scan':3}, 
             {'date':'07_12_2020','scene':'YMaze_LNovel', 'session':1,'scan':5})



In [5]:
def check_and_make_h5s(file_list,runh5=True):
    h5list, framerates = [], []
    for f in file_list:
        print(f)
        scan_str = "%s_%03d_%03d" % (f['scene'],f['session'],f['scan'])
        try:
            source_folder  = os.path.join('/media','mplitt','Backup Plus5','2P_Data')
            source_stem = os.path.join(source_folder,mouse,f['date'],f['scene'],scan_str)
            info = TwoPUtils.scanner_tools.sbx_utils.loadmat(source_stem+'.mat')
        except:
            source_folder  = os.path.join('/media','mplitt','Backup Plus3')
            source_stem = os.path.join(source_folder,mouse,f['date'],f['scene'],scan_str)
            info = TwoPUtils.scanner_tools.sbx_utils.loadmat(source_stem+'.mat')
        
        file_stem = os.path.join(write_folder,mouse,f['date'],f['scene'],scan_str)
        h5path = os.path.join(file_stem,scan_str+".h5")
        if not os.path.exists(h5path) and runh5:
            _ = TwoPUtils.scanner_tools.sbx_utils.sbx2h5(source_stem,output_name=h5path,force_2chan=True)

        h5list.append(file_stem)
        framerates.append(info['frame_rate'])
    print(h5list)
    return h5list, framerates

In [11]:
f= file_list[0]
scan_str = "%s_%03d_%03d" % (f['scene'],f['session'],f['scan'])
source_stem = os.path.join(source_folder,mouse,f['date'],f['scene'],scan_str)
file_stem = os.path.join(write_folder,mouse,f['date'],f['scene'],scan_str)
info = TwoPUtils.scanner_tools.sbx_utils.loadmat(source_stem+'.mat')
for k,v in info.items():
    print(k,v)

frame [  133   133   133 ... 34307 34307 34310]
line [135 330 492 ... 238 399  50]
event_id [1 1 1 ... 1 1 1]
resfreq 7916
postTriggerSamples 5000
recordsPerBuffer 512
bytesPerBuffer 10240000
channels 1
ballmotion []
abort_bit 0
scanbox_version 2
scanmode 1
config {'wavelength': 980, 'frames': 0, 'lines': 512, 'magnification': 3, 'magnification_list': array(['1.0', '1.2', '1.4', '1.7', '2.0', '2.4', '2.8', '3.4', '4.0',
       '4.8', '5.7', '6.7', '8.0'], dtype='<U3'), 'pmt0_gain': 0.75, 'pmt1_gain': 0.59, 'knobby': {'pos': {'x': -41.51, 'y': 70.34, 'z': 0.78, 'a': 0}, 'schedule': array([[  0,   0,  10,   0,  30],
       [  0,   0,  10,   0,  60],
       [  0,   0,  10,   0,  90],
       [  0,   0,  10,   0, 120],
       [  0,   0,  10,   0, 150],
       [  0,   0,  10,   0, 180]], dtype=uint8)}}
sz [512 796]
fold_lines 0
otwave []
otwave_um []
otparam []
otwavestyle 1
volscan 0
power_depth_link 0
opto2pow []
area_line 1
calibration [<scipy.io.matlab.mio5_params.mat_struct object at 0x

In [5]:
ops = np.load(os.path.join(write_folder,mouse,"YMazeAll","suite2p","plane0","ops.npy"),allow_pickle=True).all()
ops['save_path0'] = os.path.join(write_folder,mouse,"YMazeAll")
ops['save_path'] = os.path.join(write_folder,mouse,"YMazeAll","suite2p","plane0")
ops['ops_path'] = os.path.join(write_folder,mouse,"YMazeAll","suite2p","plane0","ops.npy")
ops['reg_file'] = os.path.join(write_folder,mouse,"YMazeAll","suite2p","plane0","data.bin")
ops['reg_file_chan2'] = os.path.join(write_folder,mouse,"YMazeAll","suite2p","plane0","data_chan2.bin")
np.save(ops['ops_path'],ops)

In [7]:
h5list_all, frs_all = check_and_make_h5s(file_list)
ops = TwoPUtils.s2p.set_ops(d={'data_path': h5list_all,
                               'save_path0': os.path.join(write_folder,mouse,"YMazeAll"),
                                'fast_disk':[],
                                'move_bin':True,
                                'two_step_registration':True,
                                'maxregshiftNR':10,
                                'nchannels':2,
                                'tau':.7,
                                'functional_chan':1,
                                'fs':frs_all[0],
                                'roidetect':True,
                                'input_format':"h5",
                                'h5py_key':'data',
                                'nchannels':2,
                                'saveNWB':True,
                                'threshold_scaling':.75,
                                'chan2_thresh':.01})
ops=s2p.run_s2p(ops=ops)

{'date': '29_11_2020', 'scene': 'YMaze_LNovel', 'session': 1, 'scan': 3}
{'date': '30_11_2020', 'scene': 'YMaze_LNovel', 'session': 1, 'scan': 7}
{'date': '01_12_2020', 'scene': 'YMaze_LNovel', 'session': 1, 'scan': 4}
{'date': '02_12_2020', 'scene': 'YMaze_LNovel', 'session': 1, 'scan': 3}
{'date': '03_12_2020', 'scene': 'YMaze_LNovel', 'session': 1, 'scan': 8}
{'date': '04_12_2020', 'scene': 'YMaze_LNovel', 'session': 1, 'scan': 2}
{'date': '05_12_2020', 'scene': 'YMaze_LNovel', 'session': 1, 'scan': 2}
{'date': '05_12_2020', 'scene': 'YMaze_RewardReversal', 'session': 1, 'scan': 4}
{'date': '06_12_2020', 'scene': 'YMaze_RewardReversal', 'session': 2, 'scan': 1}
{'date': '07_12_2020', 'scene': 'YMaze_RewardReversal', 'session': 1, 'scan': 3}
{'date': '07_12_2020', 'scene': 'YMaze_LNovel', 'session': 1, 'scan': 5}
['/mnt/BigDisk/2P_scratch/4467331.1/29_11_2020/YMaze_LNovel/YMaze_LNovel_001_003', '/mnt/BigDisk/2P_scratch/4467331.1/30_11_2020/YMaze_LNovel/YMaze_LNovel_001_007', '/mnt/Bi

In [10]:
_file_list = (#{'date': '29_11_2020', 'scene':'YMaze_LNovel','session':1,'scan':3},
             {'date': '30_11_2020', 'scene':'YMaze_LNovel','session':1,'scan':7},
             {'date': '01_12_2020', 'scene':'YMaze_LNovel','session':1,'scan':4},
             {'date': '02_12_2020', 'scene':'YMaze_LNovel','session':1,'scan':3},
             {'date': '03_12_2020', 'scene':'YMaze_LNovel','session':1,'scan':8},
             {'date': '04_12_2020', 'scene':'YMaze_LNovel','session':1,'scan':2},
            {'date': '06_12_2020', 'scene':'YMaze_RewardReversal','session':2,'scan':1})
for f in _file_list:
    
        
    _h5list,_frs = check_and_make_h5s([f,], runh5 = False)
    
    ops_dir = os.path.join(_h5list[0],'suite2p','plane0','ops.npy')
    s2p_dir = os.path.join(_h5list[0],'suite2p','plane0')
    if os.path.exists(ops_dir):
        ops = np.load(os.path.join(s2p_dir,"ops.npy"),allow_pickle=True).all()
        ops['reg_file'] = os.path.join(s2p_dir,"data.bin")
        ops['reg_file_chan2'] = os.path.join(s2p_dir,"data_chan2.bin")
        np.save(ops_dir,ops)
        
        
    ops = TwoPUtils.s2p.set_ops(d={'data_path': _h5list,
                                   'save_path0': _h5list[0],
                                    'fast_disk':[],
                                    'move_bin':True,
                                    'two_step_registration':True,
                                    'maxregshiftNR':10,
                                    'nchannels':2,
                                    'tau':.7,
                                    'functional_chan':1,
                                    'fs':_frs[0],
                                    'roidetect':True,
                                    'input_format':"h5",
                                    'threshold_scaling':1.,
                                    'h5py_key':'data',
                                    'nchannels':2})
    ops=s2p.run_s2p(ops=ops)

             

{'date': '30_11_2020', 'scene': 'YMaze_LNovel', 'session': 1, 'scan': 7}
['/mnt/BigDisk/2P_scratch/4467331.1/30_11_2020/YMaze_LNovel/YMaze_LNovel_001_007']
{}
FOUND BINARIES AND OPS IN ['/mnt/BigDisk/2P_scratch/4467331.1/30_11_2020/YMaze_LNovel/YMaze_LNovel_001_007/suite2p/plane0/ops.npy']
>>>>>>>>>>>>>>>>>>>>> PLANE 0 <<<<<<<<<<<<<<<<<<<<<<
NOTE: not running registration, plane already registered
NOTE: applying default /home/mplitt/.suite2p/classifiers/classifier_user.npy
----------- ROI DETECTION
Binning movie in chunks of length 11
Binned movie [2661,416,694], 365.37 sec.
NOTE: estimated spatial scale ~12 pixels, time epochs 2.22, threshold 22.17 
0 ROIs, score=480.23
1000 ROIs, score=38.85
Found 1372 ROIs, 82.34 sec
After removing overlaps, 1333 ROIs remain
Masks made in 29.86 sec.
>>>> CELLPOSE estimating masks in anatomical channel
ERROR importing or running cellpose, continuing without anatomical estimates
----------- Total 510.89 sec.
----------- EXTRACTION
Extracted fluorescen

In [13]:
_file_list = ({'date': '05_12_2020', 'scene':'YMaze_LNovel','session':1,'scan':2}, 
             {'date':'05_12_2020','scene':'YMaze_RewardReversal', 'session':1,'scan':4})
_h5list, _frs = check_and_make_h5s(_file_list, runh5=False)

ops_dir = os.path.join(write_folder,mouse,"05_12_2020","combined",'suite2p','plane0','ops.npy')
s2p_dir = os.path.join(write_folder,mouse,"05_12_2020","combined",'suite2p','plane0')
if os.path.exists(ops_dir):
    ops = np.load(os.path.join(s2p_dir,"ops.npy"),allow_pickle=True).all()
    ops['reg_file'] = os.path.join(s2p_dir,"data.bin")
    ops['reg_file_chan2'] = os.path.join(s2p_dir,"data_chan2.bin")
    np.save(ops_dir,ops)

ops = TwoPUtils.s2p.set_ops(d={'data_path': _h5list,
                               'save_path0': os.path.join(write_folder,mouse,"05_12_2020","combined"),
                                'fast_disk':[],
                                'move_bin':True,
                                'two_step_registration':True,
                                'maxregshiftNR':10,
                                'nchannels':2,
                                'tau':.7,
                                'functional_chan':1,
                                'fs':_frs[0],
                                'roidetect':True,
                                'input_format':"h5",
                                'threshold_scaling':1.,
                                'h5py_key':'data',
                                'nchannels':2})
ops=s2p.run_s2p(ops=ops)


 

{'date': '05_12_2020', 'scene': 'YMaze_LNovel', 'session': 1, 'scan': 2}
{'date': '05_12_2020', 'scene': 'YMaze_RewardReversal', 'session': 1, 'scan': 4}
['/mnt/BigDisk/2P_scratch/4467331.1/05_12_2020/YMaze_LNovel/YMaze_LNovel_001_002', '/mnt/BigDisk/2P_scratch/4467331.1/05_12_2020/YMaze_RewardReversal/YMaze_RewardReversal_001_004']
{}
FOUND BINARIES AND OPS IN ['/mnt/BigDisk/2P_scratch/4467331.1/05_12_2020/combined/suite2p/plane0/ops.npy']
>>>>>>>>>>>>>>>>>>>>> PLANE 0 <<<<<<<<<<<<<<<<<<<<<<
NOTE: not running registration, plane already registered
NOTE: applying default /home/mplitt/.suite2p/classifiers/classifier_user.npy
----------- ROI DETECTION
Binning movie in chunks of length 11
Binned movie [2723,416,694], 245.73 sec.
NOTE: estimated spatial scale ~12 pixels, time epochs 2.27, threshold 22.69 
0 ROIs, score=541.93
1000 ROIs, score=54.45
Found 1604 ROIs, 79.49 sec
After removing overlaps, 1535 ROIs remain
Masks made in 31.17 sec.
>>>> CELLPOSE estimating masks in anatomical chan

In [14]:
_file_list = ({'date': '07_12_2020', 'scene':'YMaze_RewardReversal','session':1,'scan':3}, 
             {'date':'07_12_2020','scene':'YMaze_LNovel', 'session':1,'scan':5})

_h5list,_frs = check_and_make_h5s(_file_list, runh5=False)

ops_dir = os.path.join(write_folder,mouse,"07_12_2020","combined",'suite2p','plane0','ops.npy')
s2p_dir = os.path.join(write_folder,mouse,"07_12_2020","combined",'suite2p','plane0')
if os.path.exists(ops_dir):
    ops = np.load(os.path.join(s2p_dir,"ops.npy"),allow_pickle=True).all()
    ops['reg_file'] = os.path.join(s2p_dir,"data.bin")
    ops['reg_file_chan2'] = os.path.join(s2p_dir,"data_chan2.bin")
    np.save(ops_dir,ops)

ops = TwoPUtils.s2p.set_ops(d={'data_path': _h5list,
                               'save_path0': os.path.join(write_folder,mouse,"07_12_2020","combined"),
                                'fast_disk':[],
                                'move_bin':True,
                                'two_step_registration':True,
                                'maxregshiftNR':10,
                                'nchannels':2,
                                'tau':.7,
                                'functional_chan':1,
                                'fs':_frs[0],
                                'roidetect':True,
                                'input_format':"h5",
                                'h5py_key':'data',
                                'nchannels':2})
ops=s2p.run_s2p(ops=ops)

{'date': '07_12_2020', 'scene': 'YMaze_RewardReversal', 'session': 1, 'scan': 3}
{'date': '07_12_2020', 'scene': 'YMaze_LNovel', 'session': 1, 'scan': 5}
['/mnt/BigDisk/2P_scratch/4467331.1/07_12_2020/YMaze_RewardReversal/YMaze_RewardReversal_001_003', '/mnt/BigDisk/2P_scratch/4467331.1/07_12_2020/YMaze_LNovel/YMaze_LNovel_001_005']
{}
FOUND BINARIES AND OPS IN ['/mnt/BigDisk/2P_scratch/4467331.1/07_12_2020/combined/suite2p/plane0/ops.npy']
>>>>>>>>>>>>>>>>>>>>> PLANE 0 <<<<<<<<<<<<<<<<<<<<<<
NOTE: not running registration, plane already registered
NOTE: applying default /home/mplitt/.suite2p/classifiers/classifier_user.npy
----------- ROI DETECTION
Binning movie in chunks of length 11
Binned movie [2674,416,694], 256.14 sec.
NOTE: estimated spatial scale ~12 pixels, time epochs 2.23, threshold 22.28 
0 ROIs, score=458.52
1000 ROIs, score=50.28
Found 1532 ROIs, 77.04 sec
After removing overlaps, 1451 ROIs remain
Masks made in 29.09 sec.
>>>> CELLPOSE estimating masks in anatomical chan

In [18]:
for h5 in h5list_all:
    !rm {h5}/*.h5 

In [13]:
!rm {h5list[-1]}/*.h5
